In [ ]:
import pythia8
import numpy as np

# Meson decay generation

In [ ]:
nevents = int(1e5)  # number of POT to generate

Z = 6          # Z for carbon (graphite target @ NuMI)
A = 12         # A for carbon

# Particles to store and files to store them in
tracked_particles = {
    111: 'pion.txt', 
    221: 'eta.txt',
    331: 'eta-prime.txt',
    15: 'tau.txt'
} 

In [ ]:
Ecmv = []

# pp event generator
pythiap = pythia8.Pythia()
pythiap.readString("Main:numberOfEvents = " + str(nevents))
# Fixed seed for now
pythiap.readString("Random:setSeed = on")
pythiap.readString("Random:seed = 1234")
# Proton on proton
pythiap.readString("Beams:idA = 2212")
pythiap.readString("Beams:idB = 2212")
# Do a safer CM frame generation (120 GeV kinetic lab energy = 15.123 GeV CM)
# Weird things happen for me in lab frame
pythiap.readString("Beams:eCM = 15.123")
# The consensus right process seems to be SoftQCD, but can consider others
#pythiap.readString("HardQCD:hardccbar = on")
#pythiap.readString("LowEnergyQCD:all = on")
pythiap.readString("SoftQCD:all = on")
# Don't put any phase space cut "by hand"
pythiap.readString("PhaseSpace:mHatMin = 0.")

# pn event generator
# Settings are largely the same, but use n instead of p
pythian = pythia8.Pythia()
pythian.readString("Main:numberOfEvents = " + str(nevents))
pythian.readString("Random:setSeed = on")
pythian.readString("Random:seed = 5678")
pythian.readString("Beams:idA = 2212")
pythian.readString("Beams:idB = 2112")
pythian.readString("Beams:eCM = 15.133")
#pythian.readString("HardQCD:hardccbar = on")
#pythian.readString("LowEnergyQCD:all = on")
pythian.readString("SoftQCD:all = on")
pythian.readString("PhaseSpace:mHatMin = 0.")
# don't decay the tracked particles
for tp in tracked_particles:
    pythiap.readString(str(tp) + ":mayDecay = off")
    pythian.readString(str(tp) + ":mayDecay = off")
    with open(tracked_particles[tp],'w') as f:
        f.write('# px\tpy\tpz\te\n')
        
# initialize and loop
pythiap.init()
pythian.init()
for i in range(nevents):
    evt = None
    # decide to do pp or pn
    if np.random.random() < float(Z)/A:
        if not pythiap.next():
            continue
        evt = pythiap.event
    else:
        if not pythian.next():
            continue
        evt = pythian.event
    # Use only if doing CM frame collisions
    pbeam = pythia8.Vec4(0,0,0,0)
    for part in evt:
        # Find the beam proton
        # event generation the CM frame: need to boost to lab frame
        if part.idAbs() == 2212 and part.status() == -12 and part.pz() > 0:
            pbeam = part.p()
        # Skip non-final
        if not part.isFinal():
            continue
        # Make sure we found the beam
        if pbeam.e() == 0.:
            raise ValueError('Beam energy vector has not been set.')
        for pdg in tracked_particles:
            if part.idAbs() == pdg:
                plab = part.p()
                plab.bst(pbeam)
                px = plab.px()
                py = plab.py()
                pz = plab.pz()
                e  = plab.e()
                with open(tracked_particles[pdg],'a') as f:
                    f.write(str(px) + '\t' + str(py) + '\t' + str(pz) + '\t' + str(e) + '\n')

# Tau generation

## Here I am attempting to get decays of taus into HNLs

In [ ]:
nevents = int(1e5)
Z = 6
A = 12

In [ ]:
# pp event generator
pythiap = pythia8.Pythia()
pythiap.readString("Main:numberOfEvents = " + str(nevents))
pythiap.readString("Random:setSeed = on")
pythiap.readString("Random:seed = 1234")
pythiap.readString("Beams:idA = 2212")
pythiap.readString("Beams:idB = 2212")
pythiap.readString("Beams:eCM = 15.123")
pythiap.readString("SoftQCD:all = on")
# In principle, hard ccbar seems better, but can't be mixed and both give D
#pythiap.readString("HardQCD:hardccbar = on")
# First turn off all SM tau decays
pythiap.readString("15:offIfAny = 16")
# Turn on decays
pythiap.readString("15:mayDecay = on")
# Decay tau in a 2 body channel with a final state sterile neutrino and charged pion 100% of the time
pythiap.readString("15:addChannel = 1 1.0 1521 18 -211")
pythiap.readString("18:m0 = 0.4")
pythiap.readString("18:mayDecay = off")
# Force the D and D_s to decay to tau
pythiap.readString("411:onMode = off")
pythiap.readString("411:onIfAny = 15")
pythiap.readString("431:onMode = off")
pythiap.readString("431:onIfAny = 15")
pythiap.readString("PhaseSpace:mHatMin = 0.")

# pn event generator
pythian = pythia8.Pythia()
pythian.readString("Main:numberOfEvents = " + str(nevents))
pythian.readString("Random:setSeed = on")
pythian.readString("Random:seed = 5678")
pythian.readString("Beams:idA = 2212")
pythian.readString("Beams:idB = 2112")
pythian.readString("Beams:eCM = 15.133")
pythian.readString("SoftQCD:all = on")
pythian.readString("15:offIfAny = 16")
pythian.readString("15:mayDecay = on")
pythian.readString("15:addChannel = 1 1.0 1521 18 -211")
pythian.readString("18:m0 = 0.4")
pythian.readString("18:mayDecay = off")
# Enhance decays to tau, but should be careful to weight appropriately
pythian.readString("411:onMode = off")
pythian.readString("411:onIfAny = 15")
pythian.readString("431:onMode = off")
pythian.readString("431:onIfAny = 15")
pythian.readString("PhaseSpace:mHatMin = 0.")

# don't decay the tracked particles
outfilename = 'hnl_new.txt'
with open(outfilename,'w') as f:
    f.write('# px\tpy\tpz\te\n')
# initialize and loop
pythiap.init()
pythian.init()
nD = 0
nDs = 0
for i in range(nevents):
    evt = None
    if np.random.random() < float(Z)/A:
        if not pythiap.next():
            continue
        evt = pythiap.event
    else:
        if not pythian.next():
            continue
        evt = pythian.event
    pbeam = pythia8.Vec4(0,0,0,0)
    for part in evt:
        # Find the beam proton
        # event generation the CM frame: need to boost to lab frame
        if part.idAbs() == 2212 and part.status() == -12 and part.pz() > 0:
            pbeam = part.p()
        if part.idAbs() == 411:
            nD += 1
        if part.idAbs() == 431:
            nDs += 1
        if not part.isFinal():
            continue
        # Make sure we found the beam
        if pbeam.e() == 0.:
            raise ValueError('Beam energy vector has not been set.')
        if part.idAbs() == 18:
            plab = part.p()
            plab.bst(pbeam)
            px = plab.px()
            py = plab.py()
            pz = plab.pz()
            e = plab.e()
            with open(outfilename,'a') as f:
                f.write(str(px) + '\t' + str(py) + '\t' + str(pz) + '\t' + str(e) + '\n')